In [2]:
from langchain.agents import Tool
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

In [27]:
from dotenv import load_dotenv
import os

load_dotenv()

google_cse_id=os.getenv("GOOGLE_CSE_ID")
google_search_api_key=os.getenv("GOOGLE_SEARCH_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from googleapiclient.discovery import build

service=build("customsearch","v1",developerKey=google_search_api_key)
res=service.cse().list(q='Python programming language',cx=google_cse_id).execute()

In [9]:
results=res.get('items',[])

for index, item in enumerate(results):
    title=item.get("title",'no title')
    link=item.get('link','no link')
    print(f"Result {index + 1}:")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print('-' * 50)

Result 1:
Title: Welcome to Python.org
Link: https://www.python.org/
--------------------------------------------------
Result 2:
Title: Python (programming language) - Wikipedia
Link: https://en.wikipedia.org/wiki/Python_(programming_language)
--------------------------------------------------
Result 3:
Title: Download Python | Python.org
Link: https://www.python.org/downloads/
--------------------------------------------------
Result 4:
Title: Introduction to Python
Link: https://www.w3schools.com/python/python_intro.asp
--------------------------------------------------
Result 5:
Title: The Python Tutorial — Python 3.13.1 documentation
Link: https://docs.python.org/3/tutorial/index.html
--------------------------------------------------
Result 6:
Title: Python as functional programming language - Python Help ...
Link: https://discuss.python.org/t/python-as-functional-programming-language/38402
--------------------------------------------------
Result 7:
Title: Python Docs
Link: http

In [ ]:
class GoogleSearch:

    def __init__(self,api_key,cse_id):

        self.service=build("customsearch","v1",developerKey=api_key)
        self.cse_id=cse_id

    def search(self,query):

        custom_prompt="""
            I have the following search results related to "{query}". Please analyze these results and provide a comprehensive, well-structured summary. Include the most relevant insights, comparisons (if applicable), and any notable trends or points. Format the summary as follows:

            1. **Key Insights**: Provide the most important takeaways from the search results.
            2. **Detailed Explanation**: Expand on the context and details provided in the results, offering a deeper understanding of the topic.
            3. **Examples or Case Studies** (if applicable): Include any examples or case studies mentioned in the results that illustrate the key points.
            4. **Conclusion**: Summarize the overall implications or significance of the findings.

            Here are the search results:
            {search_results}

            Make sure to provide a clear and concise answer with structured sections for easy reading.

        """

        response=(
            self.service.cse().list(q=query,cx=self.cse_id).execute()
        )

        search_results = response.get('items', [])
        formatted_results = ""

        for item in search_results:
            title = item.get('title', 'No Title')
            snippet = item.get('snippet', 'No Snippet')
            link = item.get('link', 'No Link')
            
            formatted_results += f"**Title**: {title}\n**Snippet**: {snippet}\n**Link**: {link}\n\n"

        prompt = custom_prompt.format(query=query, search_results=formatted_results)

        return prompt

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI

engine=GoogleSearch(api_key=google_search_api_key,cse_id=google_cse_id)

search_tool=Tool(
    name="Google Search",
    description="Retrieve information from Google Programmable Search Engine.",
    func=engine.search
)

llm=ChatGroq(model='llama-3.1-8b-instant',
             groq_api_key=groq_api_key)

# llm=ChatOpenAI(model='gpt-4o-mini',temperature=0.7)

tools=[search_tool]

agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
response=agent.run("What are the features of iphone 16 pro max?")

In [31]:
with open('iphone.txt','w') as f:
    f.write(response)

In [43]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

memory=ConversationBufferMemory(memory_key='chat_history',return_messages=True)

prompt_template = """
You are an assistant capable of browsing the internet and answering questions based on search results. You will remember prior conversations and take them into account for a more accurate response.

Here is the context of the conversation:
{chat_history}

Now, answer the following question based on your previous conversation and the current query:
{query}
"""

prompt=ChatPromptTemplate.from_template(template=prompt_template)

llm_chain=LLMChain(prompt=prompt,llm=llm)

tools=[search_tool]

agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent='chat-conversational-react-description',
    memory=memory
)


In [58]:
try:
    response = agent.invoke("Give me a very detailed and elaborate one")
except Exception as e:
    print(f"Error occurred: {e}")



The recent developments in AI are centered around generative AI, multimodal AI, and custom enterprise models. Key insights from the search results include the growing importance of AI agents, the potential of generative AI to create new content, and the increasing adoption of multimodal generative models. The global artificial intelligence market is projected to grow at a CAGR of 36.6% from 2024 to 2030, driven by the growing demand for conversational AI and gen AI solutions.

**Key Insights**:
- AI agents are the next frontier of generative AI.
- Generative AI can learn from existing artifacts to generate new, realistic artifacts.
- Multimodal generative models are on the rise.
- The global artificial intelligence market is projected to grow at a CAGR of 36.6% from 2024 to 2030.

**Detailed Explanation**:
- The integration of AI agents with virtual coworkers is expected to revolutionize complex workflows.
- Generative AI technologies are helping businesses stay competitive by automating tasks and improving decision-making.
- The conversational AI market is projected to grow from USD 13.2 billion in 2024 to USD 49.9 billion by 2030.

**Examples or Case Studies**:
- Microsoft investing $19 billion in data centers and AI chips to innovate in gen AI.
- Meta launching its largest open-source model to drive innovation in AI.

**Conclusion**:
- Generative AI, multimodal AI, and custom enterprise models are driving the recent developments in AI.
- The growing demand for conversational AI and gen AI solutions is expected to propel the global artificial intelligence market to significant growth.

In [61]:
response

{'input': 'Give me a very detailed and elaborate one',
 'chat_history': [HumanMessage(content='What are the recent developments in the world of AI', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Recent developments in the world of AI include the increasing adoption of generative AI, with more companies investing in its usage, jumping from 55% in 2023 to 75% in 2024. Additionally, multimodal AI, which can understand information from different data types, is becoming a significant trend. Other key AI trends for 2024 include AI democratization, conversational AI, predictive analytics, and agentic AI. These developments are expected to have a significant impact on various industries, including capital markets, and are transforming the AI landscape. According to various reports, including the AI Index Report 2024 and the 2024 Global Trends in AI report, AI is gaining more widespread implementation, with a focus on responsible scaling and minimizing risk. Overall, the fut

In [ ]:
response['chat_history']

[HumanMessage(content='What are the recent developments in the world of AI', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Recent developments in the world of AI include the increasing adoption of generative AI, with more companies investing in its usage, jumping from 55% in 2023 to 75% in 2024. Additionally, multimodal AI, which can understand information from different data types, is becoming a significant trend. Other key AI trends for 2024 include AI democratization, conversational AI, predictive analytics, and agentic AI. These developments are expected to have a significant impact on various industries, including capital markets, and are transforming the AI landscape. According to various reports, including the AI Index Report 2024 and the 2024 Global Trends in AI report, AI is gaining more widespread implementation, with a focus on responsible scaling and minimizing risk. Overall, the future of AI looks promising, with numerous opportunities for growth and inn